In [1]:
!rm -rf ./wide_resnet_bin_lars_4

In [2]:
%config IPCompleter.greedy=True
import numpy as np
import tensorflow as tf
import time

In [3]:
%run ./datagen.py
datagen, (x_train, y_train), (x_test, y_test) = data_preparation()

In [4]:
save_dir = './wide_resnet_bin_lars_4/'
batch_size = 100
iterations = x_train.shape[0] // batch_size
epochs = 1200
old_acc = 0
start_lr = 2.
end_lr = 0.5
decay_rate = (end_lr / start_lr) ** (1 / epochs)
k = 4

In [5]:
%run ./binary_layer.py

# resnet layer
def res_layer(inputs, filter_num, filter_size, stride, is_train,
              binarized=False, batch_norm=True, activation=True):

    x = inputs

    if binarized:
        if batch_norm:
            x = tf.layers.batch_normalization(x, training=is_train)
        if activation:
            x = tf.nn.relu(x)
        x = conv2d(inputs=x, filters=filter_num, 
                   kernel_size=filter_size, strides=stride, padding='same')
    else:
        if batch_norm:
            x = tf.layers.batch_normalization(x, training=is_train)
        if activation:
            x = tf.nn.relu(x)
        x = tf.layers.conv2d(inputs=x, filters=filter_num, 
                             kernel_size=filter_size, strides=stride, padding='same')
        
    return x

In [6]:
def wide_resnet(inputs, k, is_train, binarized):

    with tf.variable_scope("1st_Conv"):
        x = tf.layers.conv2d(inputs=inputs, filters=16, 
                             kernel_size=3, strides=1, padding='same')
        x = tf.layers.batch_normalization(x, training=is_train)
        x = tf.nn.relu(x)
        tf.summary.histogram('activation', x)

    # Res Blocks
    a = [1, 1, 1]

    for stack in range(len(a)):
        for block in range(a[stack]):

            with tf.variable_scope('ResBlock_%d_%d' % (stack+1, block+1)):

                batch_norm = True
                activation = True
                stride = 1
                filter_num = 16*k*(2**stack)
                if stack == 0:
                    if block == 0:
                        batch_norm = False
                        activation = False
                else:  
                    if block == 0:
                        stride = 2

                shortcut = x
                with tf.variable_scope('conv1'):
                    x = res_layer(x, filter_num, 3, stride, is_train, 
                                  binarized=binarized, batch_norm=batch_norm, 
                                  activation=activation)
                    
                x = tf.layers.dropout(x, 0.1)
                
                with tf.variable_scope('conv2'):
                    x = res_layer(x, filter_num, 3, 1, is_train, binarized=binarized)
                
                with tf.variable_scope('shortcut'):
                    if block == 0:
                        shortcut = res_layer(shortcut, filter_num, 1, stride, False, 
                                             binarized=False, batch_norm=False, 
                                             activation=False)
                x = x + shortcut
                tf.summary.histogram('output', x)

    with tf.variable_scope("AfterResBlock"):
        x = tf.layers.batch_normalization(x, training=is_train)
        x = tf.nn.relu(x)
        x = tf.layers.average_pooling2d(x, pool_size=8, strides=8, 
                                        padding='SAME', name='ave_pool')
        tf.summary.histogram('bn_relu_pooling', x)
    
    ######## current x.shape = (?, 1, 1, 256) ##########

    with tf.variable_scope("Flatten"):
        x = tf.transpose(x, perm=[0, 3, 1, 2])
        x = tf.layers.flatten(x)

    with tf.variable_scope("Prediction"):
        pred = tf.layers.dense(x, units=10)
        tf.summary.histogram('prediction', pred)
        
    return pred

In [7]:
tf.reset_default_graph()

with tf.device('/GPU:8'):

    inputs = tf.placeholder(tf.float32, [None, 32, 32, 3], name='input')
    outputs = tf.placeholder(tf.float32, [None, 10], name='output')
    is_train = tf.placeholder(tf.bool, name='is_train')

    global_step = tf.Variable(0, trainable=False)

    l_r = tf.train.exponential_decay(
        start_lr, global_step, iterations, decay_rate, staircase=True)
    tf.summary.scalar('learning_rate', l_r)

    opt = tf.train.MomentumOptimizer(learning_rate=l_r, momentum=0.9)

    pred = wide_resnet(inputs, k, is_train, binarized=False)

    loss = tf.losses.softmax_cross_entropy(outputs, pred)

    grads = opt.compute_gradients(loss)

    ########################## LARS ##########################
    grads_norm = [(tf.norm(grad, ord=2), tf.norm(var, ord=2)) for grad, var in grads]
    eta = 1e-4
    local_lr = [tf.where(var_norm < 1e-3, 1e-3, eta*var_norm / (grad_norm + 1e-8)) 
                for grad_norm, var_norm in grads_norm]
    new_grads = [(local_lr[i]*grad, var) for i, (grad, var) in enumerate(grads)]
    ##########################################################

    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(outputs, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    for grad, var in new_grads:
        if grad is not None:
            tf.summary.histogram(var.name.split(":")[0] + '/gradients', grad)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.control_dependencies(update_ops):
        train_op = opt.apply_gradients(new_grads, global_step=global_step)

#     kernel_vars = []

#     for i in tf.trainable_variables():
#         if 'bin/kernel' in i.name:
#             kernel_vars.append(i)

#     with tf.control_dependencies(update_ops):
#         with tf.control_dependencies([train_op]):
#             kernel_clip_op = [tf.clip_by_value(var, -1, 1) for var in kernel_vars]

    saver = tf.train.Saver(tf.global_variables())

    def add_hist(train_vars):
        for i in train_vars:
            name = i.name.split(":")[0] + '/value'
            value = i.value()
            tf.summary.histogram(name, value)

    add_hist(tf.trainable_variables())

    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    merged = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [8]:
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:

    print('*****************Training Start!*****************')
    train_writer = tf.summary.FileWriter(save_dir+'train', sess.graph)
    sess.run(tf.global_variables_initializer())

    for m in range(epochs):
        start = time.time()
        batch_gen = datagen.flow(
            x_train, y_train, batch_size=batch_size)

        for i in range(iterations):
            x_batch, y_batch = next(batch_gen)
            _, loss_train = sess.run([train_op, loss], 
                                     {inputs: x_batch, outputs: y_batch, is_train: True})
        
        summary = sess.run(merged, {inputs: x_batch, outputs: y_batch, is_train: False})
        train_writer.add_summary(summary, m*iterations + i + 1)

        val_accs = []
        for i in range(5000//(batch_size*5)):
            val_acc = sess.run(accuracy, {inputs: x_test[i*batch_size*5: (i+1)*batch_size*5],
                                          outputs: y_test[i*batch_size*5: (i+1)*batch_size*5],
                                          is_train: False})
            val_accs.append(val_acc)

        if np.mean(val_accs) > old_acc:
            old_acc = np.mean(val_accs)
            saver.save(sess, save_dir+'cifar10.ckpt')

        end = time.time()
        print('Epoch: {}'.format(m + 1),
              'Train_loss: {:.3f}'.format(loss_train),
              'Val_acc: {:.3f}'.format(np.mean(val_accs)),
              'Time consumed: {:.4f} s'.format(end - start))

    print('*****************Training End!*****************')

*****************Training Start!*****************
Epoch: 1 Train_loss: 1.335 Val_acc: 0.482 Time consumed: 42.5382 s
Epoch: 2 Train_loss: 1.242 Val_acc: 0.520 Time consumed: 37.9954 s
Epoch: 3 Train_loss: 0.978 Val_acc: 0.463 Time consumed: 37.2894 s
Epoch: 4 Train_loss: 1.105 Val_acc: 0.580 Time consumed: 37.5623 s
Epoch: 5 Train_loss: 0.927 Val_acc: 0.573 Time consumed: 36.4903 s
Epoch: 6 Train_loss: 0.970 Val_acc: 0.630 Time consumed: 36.4912 s
Epoch: 7 Train_loss: 0.756 Val_acc: 0.637 Time consumed: 36.5342 s
Epoch: 8 Train_loss: 0.789 Val_acc: 0.646 Time consumed: 36.7843 s
Epoch: 9 Train_loss: 0.765 Val_acc: 0.684 Time consumed: 36.7459 s
Epoch: 10 Train_loss: 0.758 Val_acc: 0.628 Time consumed: 36.5305 s
Epoch: 11 Train_loss: 0.682 Val_acc: 0.683 Time consumed: 36.5431 s
Epoch: 12 Train_loss: 0.702 Val_acc: 0.658 Time consumed: 36.4787 s
Epoch: 13 Train_loss: 0.620 Val_acc: 0.715 Time consumed: 36.7819 s
Epoch: 14 Train_loss: 0.604 Val_acc: 0.723 Time consumed: 36.7853 s
Epoch: 

Epoch: 121 Train_loss: 0.119 Val_acc: 0.824 Time consumed: 48.3239 s
Epoch: 122 Train_loss: 0.143 Val_acc: 0.833 Time consumed: 54.0956 s
Epoch: 123 Train_loss: 0.271 Val_acc: 0.839 Time consumed: 54.1814 s
Epoch: 124 Train_loss: 0.146 Val_acc: 0.812 Time consumed: 47.6146 s
Epoch: 125 Train_loss: 0.207 Val_acc: 0.836 Time consumed: 47.7535 s
Epoch: 126 Train_loss: 0.160 Val_acc: 0.820 Time consumed: 47.9668 s
Epoch: 127 Train_loss: 0.092 Val_acc: 0.831 Time consumed: 47.6191 s
Epoch: 128 Train_loss: 0.173 Val_acc: 0.825 Time consumed: 48.0212 s
Epoch: 129 Train_loss: 0.139 Val_acc: 0.826 Time consumed: 47.9486 s
Epoch: 130 Train_loss: 0.179 Val_acc: 0.847 Time consumed: 47.1626 s
Epoch: 131 Train_loss: 0.084 Val_acc: 0.860 Time consumed: 50.3926 s
Epoch: 132 Train_loss: 0.222 Val_acc: 0.826 Time consumed: 47.4529 s
Epoch: 133 Train_loss: 0.117 Val_acc: 0.820 Time consumed: 46.2659 s
Epoch: 134 Train_loss: 0.174 Val_acc: 0.828 Time consumed: 47.5043 s
Epoch: 135 Train_loss: 0.147 Val_a

Epoch: 240 Train_loss: 0.028 Val_acc: 0.865 Time consumed: 36.5363 s
Epoch: 241 Train_loss: 0.146 Val_acc: 0.835 Time consumed: 36.5194 s
Epoch: 242 Train_loss: 0.028 Val_acc: 0.860 Time consumed: 36.6460 s
Epoch: 243 Train_loss: 0.021 Val_acc: 0.859 Time consumed: 36.8581 s
Epoch: 244 Train_loss: 0.109 Val_acc: 0.872 Time consumed: 36.9993 s
Epoch: 245 Train_loss: 0.241 Val_acc: 0.841 Time consumed: 36.4638 s
Epoch: 246 Train_loss: 0.121 Val_acc: 0.848 Time consumed: 36.0683 s
Epoch: 247 Train_loss: 0.072 Val_acc: 0.854 Time consumed: 36.3321 s
Epoch: 248 Train_loss: 0.186 Val_acc: 0.834 Time consumed: 35.9405 s
Epoch: 249 Train_loss: 0.085 Val_acc: 0.820 Time consumed: 36.4209 s
Epoch: 250 Train_loss: 0.074 Val_acc: 0.866 Time consumed: 36.2691 s
Epoch: 251 Train_loss: 0.034 Val_acc: 0.872 Time consumed: 37.0575 s
Epoch: 252 Train_loss: 0.069 Val_acc: 0.848 Time consumed: 36.2007 s
Epoch: 253 Train_loss: 0.076 Val_acc: 0.866 Time consumed: 36.3784 s
Epoch: 254 Train_loss: 0.077 Val_a

Epoch: 359 Train_loss: 0.015 Val_acc: 0.854 Time consumed: 36.4337 s
Epoch: 360 Train_loss: 0.036 Val_acc: 0.875 Time consumed: 36.9622 s
Epoch: 361 Train_loss: 0.050 Val_acc: 0.878 Time consumed: 36.7665 s
Epoch: 362 Train_loss: 0.024 Val_acc: 0.857 Time consumed: 36.4275 s
Epoch: 363 Train_loss: 0.024 Val_acc: 0.867 Time consumed: 36.4771 s
Epoch: 364 Train_loss: 0.022 Val_acc: 0.873 Time consumed: 36.4723 s
Epoch: 365 Train_loss: 0.041 Val_acc: 0.874 Time consumed: 36.3366 s
Epoch: 366 Train_loss: 0.038 Val_acc: 0.876 Time consumed: 36.5289 s
Epoch: 367 Train_loss: 0.057 Val_acc: 0.865 Time consumed: 36.5275 s
Epoch: 368 Train_loss: 0.073 Val_acc: 0.869 Time consumed: 36.2645 s
Epoch: 369 Train_loss: 0.050 Val_acc: 0.861 Time consumed: 36.5188 s
Epoch: 370 Train_loss: 0.010 Val_acc: 0.875 Time consumed: 36.4650 s
Epoch: 371 Train_loss: 0.062 Val_acc: 0.861 Time consumed: 36.5641 s
Epoch: 372 Train_loss: 0.179 Val_acc: 0.856 Time consumed: 36.8426 s
Epoch: 373 Train_loss: 0.002 Val_a

Epoch: 478 Train_loss: 0.062 Val_acc: 0.872 Time consumed: 36.4793 s
Epoch: 479 Train_loss: 0.064 Val_acc: 0.855 Time consumed: 37.3087 s
Epoch: 480 Train_loss: 0.006 Val_acc: 0.870 Time consumed: 37.2726 s
Epoch: 481 Train_loss: 0.037 Val_acc: 0.881 Time consumed: 37.3859 s
Epoch: 482 Train_loss: 0.027 Val_acc: 0.861 Time consumed: 37.7959 s
Epoch: 483 Train_loss: 0.018 Val_acc: 0.853 Time consumed: 37.6235 s
Epoch: 484 Train_loss: 0.002 Val_acc: 0.883 Time consumed: 37.0957 s
Epoch: 485 Train_loss: 0.029 Val_acc: 0.884 Time consumed: 36.4976 s
Epoch: 486 Train_loss: 0.017 Val_acc: 0.849 Time consumed: 36.6963 s
Epoch: 487 Train_loss: 0.096 Val_acc: 0.830 Time consumed: 36.8762 s
Epoch: 488 Train_loss: 0.011 Val_acc: 0.884 Time consumed: 36.8336 s
Epoch: 489 Train_loss: 0.017 Val_acc: 0.864 Time consumed: 36.7922 s
Epoch: 490 Train_loss: 0.017 Val_acc: 0.867 Time consumed: 36.7956 s
Epoch: 491 Train_loss: 0.076 Val_acc: 0.878 Time consumed: 37.2290 s
Epoch: 492 Train_loss: 0.007 Val_a

...

Epoch: 716 Train_loss: 0.042 Val_acc: 0.883 Time consumed: 36.2241 s
Epoch: 717 Train_loss: 0.000 Val_acc: 0.869 Time consumed: 36.4106 s
Epoch: 718 Train_loss: 0.022 Val_acc: 0.869 Time consumed: 36.2286 s
Epoch: 719 Train_loss: 0.080 Val_acc: 0.869 Time consumed: 36.1829 s
Epoch: 720 Train_loss: 0.020 Val_acc: 0.871 Time consumed: 36.3779 s
Epoch: 721 Train_loss: 0.009 Val_acc: 0.878 Time consumed: 36.2285 s
Epoch: 722 Train_loss: 0.028 Val_acc: 0.873 Time consumed: 36.4032 s
Epoch: 723 Train_loss: 0.045 Val_acc: 0.854 Time consumed: 36.2888 s
Epoch: 724 Train_loss: 0.001 Val_acc: 0.869 Time consumed: 36.3171 s
Epoch: 725 Train_loss: 0.123 Val_acc: 0.870 Time consumed: 36.5025 s
Epoch: 726 Train_loss: 0.003 Val_acc: 0.874 Time consumed: 36.0656 s
Epoch: 727 Train_loss: 0.091 Val_acc: 0.880 Time consumed: 36.5445 s
Epoch: 728 Train_loss: 0.003 Val_acc: 0.871 Time consumed: 36.8784 s
Epoch: 729 Train_loss: 0.029 Val_acc: 0.867 Time consumed: 36.9229 s
Epoch: 730 Train_loss: 0.039 Val_a

Epoch: 835 Train_loss: 0.000 Val_acc: 0.882 Time consumed: 37.2449 s
Epoch: 836 Train_loss: 0.000 Val_acc: 0.865 Time consumed: 37.5761 s
Epoch: 837 Train_loss: 0.112 Val_acc: 0.872 Time consumed: 37.4065 s
Epoch: 838 Train_loss: 0.000 Val_acc: 0.872 Time consumed: 37.3959 s
Epoch: 839 Train_loss: 0.003 Val_acc: 0.876 Time consumed: 37.9029 s
Epoch: 840 Train_loss: 0.000 Val_acc: 0.873 Time consumed: 37.7133 s
Epoch: 841 Train_loss: 0.110 Val_acc: 0.864 Time consumed: 37.8234 s
Epoch: 842 Train_loss: 0.009 Val_acc: 0.893 Time consumed: 39.5374 s
Epoch: 843 Train_loss: 0.005 Val_acc: 0.881 Time consumed: 37.6146 s
Epoch: 844 Train_loss: 0.009 Val_acc: 0.867 Time consumed: 37.0221 s
Epoch: 845 Train_loss: 0.000 Val_acc: 0.869 Time consumed: 37.1841 s
Epoch: 846 Train_loss: 0.000 Val_acc: 0.881 Time consumed: 36.9181 s
Epoch: 847 Train_loss: 0.064 Val_acc: 0.883 Time consumed: 36.8053 s
Epoch: 848 Train_loss: 0.007 Val_acc: 0.868 Time consumed: 36.8307 s
Epoch: 849 Train_loss: 0.005 Val_a

Epoch: 954 Train_loss: 0.002 Val_acc: 0.882 Time consumed: 37.4989 s
Epoch: 955 Train_loss: 0.130 Val_acc: 0.886 Time consumed: 37.6782 s
Epoch: 956 Train_loss: 0.007 Val_acc: 0.880 Time consumed: 37.6214 s
Epoch: 957 Train_loss: 0.001 Val_acc: 0.886 Time consumed: 37.8930 s
Epoch: 958 Train_loss: 0.001 Val_acc: 0.884 Time consumed: 38.0638 s
Epoch: 959 Train_loss: 0.000 Val_acc: 0.885 Time consumed: 38.3696 s
Epoch: 960 Train_loss: 0.001 Val_acc: 0.875 Time consumed: 38.1624 s
Epoch: 961 Train_loss: 0.000 Val_acc: 0.870 Time consumed: 37.5044 s
Epoch: 962 Train_loss: 0.000 Val_acc: 0.870 Time consumed: 37.7791 s
Epoch: 963 Train_loss: 0.000 Val_acc: 0.883 Time consumed: 37.9446 s
Epoch: 964 Train_loss: 0.006 Val_acc: 0.882 Time consumed: 37.5160 s
Epoch: 965 Train_loss: 0.039 Val_acc: 0.877 Time consumed: 38.1167 s
Epoch: 966 Train_loss: 0.000 Val_acc: 0.875 Time consumed: 37.6735 s
Epoch: 967 Train_loss: 0.000 Val_acc: 0.872 Time consumed: 37.1767 s
Epoch: 968 Train_loss: 0.018 Val_a

Epoch: 1072 Train_loss: 0.002 Val_acc: 0.889 Time consumed: 36.1782 s
Epoch: 1073 Train_loss: 0.000 Val_acc: 0.884 Time consumed: 36.4515 s
Epoch: 1074 Train_loss: 0.025 Val_acc: 0.886 Time consumed: 36.0661 s
Epoch: 1075 Train_loss: 0.000 Val_acc: 0.882 Time consumed: 36.2184 s
Epoch: 1076 Train_loss: 0.001 Val_acc: 0.888 Time consumed: 36.1251 s
Epoch: 1077 Train_loss: 0.000 Val_acc: 0.890 Time consumed: 36.0875 s
Epoch: 1078 Train_loss: 0.000 Val_acc: 0.881 Time consumed: 36.4684 s
Epoch: 1079 Train_loss: 0.304 Val_acc: 0.882 Time consumed: 36.2737 s
Epoch: 1080 Train_loss: 0.000 Val_acc: 0.886 Time consumed: 36.1242 s
Epoch: 1081 Train_loss: 0.056 Val_acc: 0.886 Time consumed: 36.2253 s
Epoch: 1082 Train_loss: 0.001 Val_acc: 0.879 Time consumed: 35.9680 s
Epoch: 1083 Train_loss: 0.000 Val_acc: 0.878 Time consumed: 35.6147 s
Epoch: 1084 Train_loss: 0.000 Val_acc: 0.890 Time consumed: 35.8840 s
Epoch: 1085 Train_loss: 0.000 Val_acc: 0.883 Time consumed: 35.7956 s
Epoch: 1086 Train_lo

Epoch: 1190 Train_loss: 0.001 Val_acc: 0.894 Time consumed: 36.6396 s
Epoch: 1191 Train_loss: 0.000 Val_acc: 0.880 Time consumed: 36.7833 s
Epoch: 1192 Train_loss: 0.000 Val_acc: 0.887 Time consumed: 36.8786 s
Epoch: 1193 Train_loss: 0.000 Val_acc: 0.888 Time consumed: 36.8372 s
Epoch: 1194 Train_loss: 0.000 Val_acc: 0.885 Time consumed: 36.8696 s
Epoch: 1195 Train_loss: 0.000 Val_acc: 0.888 Time consumed: 36.8388 s
Epoch: 1196 Train_loss: 0.042 Val_acc: 0.878 Time consumed: 36.9318 s
Epoch: 1197 Train_loss: 0.000 Val_acc: 0.887 Time consumed: 37.0130 s
Epoch: 1198 Train_loss: 0.000 Val_acc: 0.883 Time consumed: 36.8138 s
Epoch: 1199 Train_loss: 0.000 Val_acc: 0.885 Time consumed: 36.5383 s
Epoch: 1200 Train_loss: 0.000 Val_acc: 0.890 Time consumed: 36.7159 s
*****************Training End!*****************
